In [45]:
import torch
import os

In [9]:
#only in google colab
!git clone https://github.com/Giordano-Cicchetti/MaskTune_NN

^C


Cloning into 'MaskTune_NN'...


In [46]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [47]:
from torchvision.transforms import ToTensor
from biased_mnist import *

train_data_RGB = BiasedMNIST(
    root = 'data',
    train = True,                         
    transform = ToTensor(), 
    download = True,            
)

data\MNISTRGB_BIASED\train


In [32]:
train_data_RGB.data_new[59900]

In [48]:
from torch.utils.data import DataLoader

loaders = {
    'train' : torch.utils.data.DataLoader(train_data_RGB, 
                                          batch_size=128, 
                                          shuffle=True, 
                                          num_workers=1)
}
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x2164b1e08b0>}

In [49]:
from train import *
train= TrainBaseERM(device)

#train.run_an_epoch(loaders['train'],0)

In [50]:
train.run_an_epoch(loaders['train'],1,device=device)

train epoch 1:   0%|          | 0/469 [00:11<?, ?it/s]


RuntimeError: "nll_loss_forward_reduce_cuda_kernel_2d_index" not implemented for 'Int'

In [8]:
train.run_an_epoch(loaders['train'],2)

train epoch 2:   0%|          | 0/469 [00:05<?, ?it/s]


RuntimeError: expected scalar type Long but found Int

In [ ]:
for i in range(10):
  train.run_an_epoch(loaders['train'],i)

train epoch 9: 100%|██████████| 469/469 [01:18<00:00,  5.98it/s, loss=0.56, accuracy=0.709]


In [ ]:
!rm data -r